<h1>snRNA-seq re-analysis

In [ ]:
# original raw data acquired by Schirmer et al. (Nature, 2019)
# NCBI Sequence Read Archive (SRA), accession number PRJNA544731

# load libraries
library(tidyverse)
library(Seurat)
library(Polychrome)

# import UMI matrix from Cell Ranger
matrix_dir = "XXX" # insert path to UMI matrix here
data_raw = Read10X(data.dir = matrix_dir)
data = CreateSeuratObject(counts = data_raw, project = "snRNA-seq", min.cells = 3, min.features = 200)

# calculate the percentage of mitochondrial features
mito.features = grep(pattern = "^MT-", x = rownames(x = data), value = TRUE)
percent.mito = Matrix::colSums(x = GetAssayData(object = data, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = data, slot = 'counts'))
data[['percent.mito']] = percent.mito # store information

# filter out cells that have unique feature counts over 5000 or less than 200
data = subset(data, subset = nFeature_RNA > 500 & nCount_RNA > 1000)

# Normalizing the data
data = NormalizeData(data, normalization.method = "LogNormalize", scale.factor = 1e4)

# detection of variable genes across the single cells
data <- FindVariableFeatures(data, selection.method = 'mean.var.plot', mean.cutoff = c(0.0125, 3), dispersion.cutoff = c(0.5, Inf))

# remove MT genes
markers.remove <- grep(pattern = "^MT-",  x = rownames(data), value = TRUE)

data@assays$RNA@var.features <- data@assays$RNA@var.features[!(data@assays$RNA@var.features%in%markers.remove)]

# scaling the data and removing unwanted sources of variation
data = ScaleData(data)

# perform linear dimensional reduction
data = RunPCA(data, features = VariableFeatures(data), verbose = FALSE)
# examine and visualize PCA results a few different ways
print(x = data[['pca']], dims = 1:5, nfeatures = 5, projected = FALSE)

# get batches based on cell names
samples_batches = sapply(colnames(GetAssayData(data, slot = "counts")),
                      FUN=function(x){substr(x,18,19)})

# turn to numbers and add cell names to them
samples_batches = as.numeric(as.factor(samples_batches))
names(samples_batches) = colnames(GetAssayData(data, slot = "counts"))

# plot genes per cell
complexity.per.cell = apply(GetAssayData(data, slot = "counts"),
                             2, function(x) sum(x>0))
plot(complexity.per.cell ~ jitter(samples_batches,2))

sample.effect = samples_batches
data = AddMetaData(data, sample.effect, "sample.effect")

for (i in 1:length(x = data.list)) {
    data.list[[i]] = NormalizeData(object = data.list[[i]], verbose = FALSE)
    data.list[[i]] = FindVariableFeatures(object = data.list[[i]], 
        selection.method = "vst", nfeatures = 2000, verbose = FALSE)
}

reference.list <- data.list
data.anchors <- FindIntegrationAnchors(object.list = reference.list, dims = 1:30)

data.integrated = IntegrateData(anchorset = data.anchors, dims = 1:30)

# switch to integrated assay
DefaultAssay(object = data.integrated) <- "integrated"

# Scaling the data and removing unwanted sources of variation
data.integrated <- ScaleData(object = data.integrated)

# perform linear dimensional reduction
data.integrated <- RunPCA(object = data.integrated, npcs = 30, verbose = FALSE)
# examine and visualize PCA results a few different ways
print(x = data.integrated[['pca']], dims = 1:5, nfeatures = 5, projected = FALSE)
VizDimLoadings(object = data.integrated, dims = 1:2)
DimPlot(object = data.integrated)
DimPlot(object = data.integrated, group.by="sample.effect")

# dimensional reduction and clustering
data = FindNeighbors(data, dims = 1:16)
data = FindClusters(data, resolution = 0.8)

# perform non-linear dimensional reduction (tSNE)
data = RunTSNE(data, dims = 1:16)

# visualize data
data(palette36) # color palette for clusters
colpal = palette36[1:33]
colpal = colpal %>% as.vector()

# tSNE plot and clustering
Idents(data) <- "RNA_snn_res.0.8"
DimPlot(data, reduction = 'tsne', cols = colpal, label = F, label.size = 5, pt.size = 1) + 
  theme_void() + NoLegend()
ggsave('output/tsne.png', plot = last_plot(), scale = 1, dpi = 900)
DimPlot(data, reduction = 'tsne', cols = colpal, label = T, label.size = 5, pt.size = 1) + 
  theme_void()
ggsave('output/tsne.pdf', plot = last_plot(), scale = 1)

# signature genes for clusters (as used by Schirmer et al. for cell type identification)
sig_genes = c('SYT1', 'SLC17A7', 'GAD2', 'AQP4', 'PLP1', 'P2RY12', 'CSF1R', 'CSF1', 'IL34')
for (i in 1:length(sig_genes)){
    FeaturePlot(data, features = sig_genes[i], label.size = 5, cols = c("lightgrey", "red3"), pt.size = 1.5) +
        theme_void() + NoLegend() + theme(title = element_blank())
    ggsave(paste0('output/', 'tsne_', tolower(sig_genes[i]), '.png'), plot = last_plot(), scale = 1, dpi = 900, bg = "transparent")
}

# CSF1R, CSF1 and IL34 in tsne
goi = c('CSF1R', 'IL34', 'CSF1') # define genes of interest
for (i in 1:length(goi)){
    FeaturePlot(data, features = goi[i], label.size = 5, cols = c("lightgrey", "red3"), pt.size = 1.5) +
        theme_void() + NoLegend() + theme(title = element_blank())
    ggsave(paste0('output/', 'tsne_', tolower(goi[i]), '.png'), plot = last_plot(), scale = 1, dpi = 900)
}

# cluster-specific expression of CSF1R, CSF1 and IL34
goi = c('CSF1R', 'IL34', 'CSF1') # define genes of interest
for (i in 1:length(goi)){
    Idents(data) <- "RNA_snn_res.0.8"
    DotPlot(data, features = goi[i], dot.scale = 3, cols = c('red3', 'red3'), split.by = "diagnosis") +
      theme(text=element_text(size=6,  family="sans"),
        axis.text.x = element_text(angle=45, vjust=1, hjust=1, size=6, family="sans"),
        axis.text.y = element_text(size=6, family="sans", hjust = 1, vjust = 0.35),
        axis.line = element_line(colour = 'black', size = 0.2),
        axis.ticks = element_line(colour = 'black', size = 0.2)
        )
    ggsave(paste0('output/', 'dotplot_', tolower(goi[i]), '_msctrl.pdf'), plot = last_plot(), width = 6, height = 16, units = "cm")
}